# Google SQL for PostgreSQL

> [Google Cloud SQL](https://cloud.google.com/sql) 是一个完全托管的关系型数据库服务，具备高性能、无缝集成和卓越的可扩展性。它提供 `MySQL`、`PostgreSQL` 和 `SQL Server` 数据库引擎。通过 Langchain 集成 Cloud SQL，您可以扩展数据库应用以构建赋能 AI 的体验。

本 Notebook 将介绍如何使用 `Google Cloud SQL for PostgreSQL` 类来存储聊天消息历史记录。

在 [GitHub](https://github.com/googleapis/langchain-google-cloud-sql-pg-python/) 上了解更多关于该包的信息。

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/googleapis/langchain-google-cloud-sql-pg-python/blob/main/docs/chat_message_history.ipynb)

## 开始之前

要运行此笔记本，您需要执行以下操作：

 * [创建 Google Cloud 项目](https://developers.google.com/workspace/guides/create-project)
 * [启用 Cloud SQL Admin API](https://console.cloud.google.com/marketplace/product/google/sqladmin.googleapis.com)
 * [创建 Cloud SQL for PostgreSQL 实例](https://cloud.google.com/sql/docs/postgres/create-instance)
 * [创建 Cloud SQL 数据库](https://cloud.google.com/sql/docs/mysql/create-manage-databases)
 * [为数据库添加 IAM 数据库用户](https://cloud.google.com/sql/docs/postgres/add-manage-iam-users#creating-a-database-user)（可选）

### 🦜🔗 库安装
集成位于其自身的 `langchain-google-cloud-sql-pg` 包中，因此我们需要安装它。

In [ ]:
%pip install --upgrade --quiet langchain-google-cloud-sql-pg langchain-google-vertexai

**仅限 Colab：** 取消注释以下单元格以重启内核，或使用按钮重启内核。对于 Vertex AI Workbench，您可以使用顶部的按钮重启终端。

In [ ]:
# # Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

### 🔐 身份验证
使用此笔记本登录的 IAM 用户身份验证到 Google Cloud，以访问您的 Google Cloud 项目。

* 如果您正在使用 Colab 运行此笔记本，请使用下面的单元格并继续。
* 如果您正在使用 Vertex AI Workbench，请在此处查看设置说明：[https://github.com/GoogleCloudPlatform/generative-ai/tree/main/setup-env](https://github.com/GoogleCloudPlatform/generative-ai/tree/main/setup-env)。

In [1]:
from google.colab import auth

auth.authenticate_user()

### ☁ 设置您的 Google Cloud 项目
设置您的 Google Cloud 项目，以便在此 notebook 中利用 Google Cloud 资源。

如果您不知道您的项目 ID，可以尝试以下方法：

* 运行 `gcloud config list`。
* 运行 `gcloud projects list`。
* 查看支持页面：[查找项目 ID](https://support.google.com/googleapi/answer/7014113)。

In [ ]:
# @markdown Please fill in the value below with your Google Cloud project ID and then run the cell.

PROJECT_ID = "my-project-id"  # @param {type:"string"}

# Set the project id
!gcloud config set project {PROJECT_ID}

### 💡 API 启用
`langchain-google-cloud-sql-pg` 包要求您在 Google Cloud 项目中[启用 Cloud SQL Admin API](https://console.cloud.google.com/flows/enableapi?apiid=sqladmin.googleapis.com)。

In [3]:
# enable Cloud SQL Admin API
!gcloud services enable sqladmin.googleapis.com

## 基本用法

### 设置 Cloud SQL 数据库值
在 [Cloud SQL 实例页面](https://console.cloud.google.com/sql?_ga=2.223735448.2062268965.1707700487-2088871159.1707257687) 中查找您的数据库值。

In [4]:
# @title Set Your Values Here { display-mode: "form" }
REGION = "us-central1"  # @param {type: "string"}
INSTANCE = "my-postgresql-instance"  # @param {type: "string"}
DATABASE = "my-database"  # @param {type: "string"}
TABLE_NAME = "message_store"  # @param {type: "string"}

### PostgresEngine 连接池

将 Cloud SQL 建立为 ChatMessageHistory 内存存储的要求和论据之一是 `PostgresEngine` 对象。`PostgresEngine` 配置到 Cloud SQL 数据库的连接池，从而能够从您的应用程序成功建立连接并遵循行业最佳实践。

要使用 `PostgresEngine.from_instance()` 创建 `PostgresEngine`，您只需要提供 4 项内容：

1.  `project_id`：Cloud SQL 实例所在的 Google Cloud 项目的项目 ID。
1.  `region`：Cloud SQL 实例所在的区域。
1.  `instance`：Cloud SQL 实例的名称。
1.  `database`：要连接到 Cloud SQL 实例上的数据库的名称。

默认情况下，将使用 [IAM 数据库身份验证](https://cloud.google.com/sql/docs/postgres/iam-authentication#iam-db-auth) 作为数据库身份验证方法。此库使用属于从环境变量加载的 [应用程序默认凭据 (ADC)](https://cloud.google.com/docs/authentication/application-default-credentials) 的 IAM 主体。

有关 IAM 数据库身份验证的更多信息，请参阅：

*   [为 IAM 数据库身份验证配置实例](https://cloud.google.com/sql/docs/postgres/create-edit-iam-instances)
*   [使用 IAM 数据库身份验证管理用户](https://cloud.google.com/sql/docs/postgres/add-manage-iam-users)

此外，还可以选择使用 [内置数据库身份验证](https://cloud.google.com/sql/docs/postgres/built-in-authentication)，通过用户名和密码访问 Cloud SQL 数据库。只需向 `PostgresEngine.from_instance()` 提供可选的 `user` 和 `password` 参数：

*   `user`：用于内置数据库身份验证和登录的数据库用户
*   `password`：用于内置数据库身份验证和登录的数据库密码。

In [5]:
from langchain_google_cloud_sql_pg import PostgresEngine

engine = PostgresEngine.from_instance(
    project_id=PROJECT_ID, region=REGION, instance=INSTANCE, database=DATABASE
)

### 初始化表
`PostgresChatMessageHistory` 类需要一个具有特定架构的数据库表来存储聊天消息历史记录。

`PostgresEngine` 引擎有一个名为 `init_chat_history_table()` 的辅助方法，可用于为您创建具有适当架构的表。

In [ ]:
engine.init_chat_history_table(table_name=TABLE_NAME)

### PostgresChatMessageHistory

要初始化 `PostgresChatMessageHistory` 类，您只需要提供 3 个内容：

1. `engine` - `PostgresEngine` 引擎的实例。
1. `session_id` - 一个唯一的标识符字符串，用于指定会话的 ID。
1. `table_name` : 用于存储聊天消息历史的 Cloud SQL 数据库中的表名。

In [10]:
from langchain_google_cloud_sql_pg import PostgresChatMessageHistory

history = PostgresChatMessageHistory.create_sync(
    engine, session_id="test_session", table_name=TABLE_NAME
)
history.add_user_message("hi!")
history.add_ai_message("whats up?")

In [11]:
history.messages

[HumanMessage(content='hi!'), AIMessage(content='whats up?')]

#### 清理
当特定会话的历史记录已过时且可以删除时，可以按以下方式进行。

**注意：** 删除后，数据将不再存储在 Cloud SQL 中，并且将永久丢失。

In [12]:
history.clear()

## 🔗 链式调用

我们可以轻松地将此消息历史记录类与 [LCEL Runnable](/docs/how_to/message_history) 结合使用

为此，我们将使用 [Google 的 Vertex AI 聊天模型之一](/docs/integrations/chat/google_vertex_ai_palm)，这需要你在 Google Cloud 项目中[启用 Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com)。

In [13]:
# enable Vertex AI API
!gcloud services enable aiplatform.googleapis.com

In [14]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_google_vertexai import ChatVertexAI

In [15]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

chain = prompt | ChatVertexAI(project=PROJECT_ID)

In [16]:
chain_with_history = RunnableWithMessageHistory(
    chain,
    lambda session_id: PostgresChatMessageHistory.create_sync(
        engine,
        session_id=session_id,
        table_name=TABLE_NAME,
    ),
    input_messages_key="question",
    history_messages_key="history",
)

In [17]:
# This is where we configure the session id
config = {"configurable": {"session_id": "test_session"}}

In [18]:
chain_with_history.invoke({"question": "Hi! I'm bob"}, config=config)

AIMessage(content=' Hello Bob, how can I help you today?')

In [19]:
chain_with_history.invoke({"question": "Whats my name"}, config=config)

AIMessage(content=' Your name is Bob.')